In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Local imports
from data_fetcher import DataFetcher
# from preprocessing import Processor

In [3]:
# Define some string constants for easy typing
SAMPLE_DATA_BY_SITE = 'sampleData/bySite'
SAMPLE_DATA_BY_COUNTY = 'sampleData/byCounty'
SAMPLE_DATA_BY_STATE = 'sampleData/byState'
SAMPLE_DATA_BY_BOX = 'sampleData/byBox'
SAMPLE_DATA_BY_CBSA = 'sampleData/byCBSA'

LIST_STATES = 'list/states'
LIST_COUNTIES_BY_STATE = 'list/countiesByState'
LIST_SITES_BY_COUNTY = 'list/sitesByCounty'
LIST_CBSAs = 'list/cbsas'
LIST_PARAM_CLASSES = 'list/classes'
LIST_PARAM_IN_CLASS = 'list/parametersByClass'

In [4]:
datafetcher = DataFetcher()

In [5]:
# Example calls to list codes
cali_code = datafetcher.get_codes(LIST_STATES, all=False, value='California')
print('California state code:', cali_code)

criteria_code = datafetcher.get_codes(LIST_PARAM_CLASSES, all=False, value='Criteria Pollutants')
print('Criteria polutants code:', criteria_code)

carbon_monoxide_code = datafetcher.get_codes(LIST_PARAM_IN_CLASS, all=False, value='Carbon monoxide', nparams={'pc':criteria_code})
print('Carbon monoxide code:', carbon_monoxide_code)

California state code: 06
Criteria polutants code: CRITERIA
Carbon monoxide code: 42101


In [6]:
datafetcher.all_codes

,value_represented
code,
11101,Suspended particulate (TSP)
11102,Suspended particulate (TSP) LC
11103,Benzene soluble organics (TSP)
11104,Total polynuclear hydrocarbons
11114,Windblown particulate
...,...
88500,PM2.5 Total Atmospheric
88501,PM2.5 Raw Data
88502,Acceptable PM2.5 AQI & Speciation Mass


In [7]:
_ = datafetcher.find_code('Cristabalite', verbose=True)

Cristabalite code is: 11122


## Explore data by area codes to find good location for modelling

We want to find a site, or small grroup of sites, that have enough data for us to train a useeful model. We need this set to contain metereological, ozone, particulate matter, and pollutant data.

We sample a day per year for 5 of the last 20 years and find the site with the best data for this particular county/state pair. We do this sampling because the API will lock us out if we try to get too much yearly data.

This code takes roughly 30 minutes to run. And it finds that Los Angels-North Main Street had the most data.

In [8]:
# r = datafetcher.find_best_location()
# with open('data.json', 'w') as fp:
#     json.dump(r, fp)

with open('data.json', 'r') as f:
  r = json.load(f)

Let's find which sites have the most data

In [9]:
data = r['Data']
metadata = r['Metadata']

num_codes = len(data['Azusa']) # Any key in result dict
num_years = len(data['Azusa'][0])

yearly_results = {site:[sum([data[site][code][year] for code in range(num_codes)]) for year in range(num_years)] for site in data}
yearly_best = {site: ([date for date, val in enumerate(yearly_results[site]) if val == max(yearly_results[site])], max(yearly_results[site])) for site in yearly_results}
yearly_best_sorted = sorted(yearly_best.items(), key=lambda x : x[1][1], reverse=True)
yearly_best_sorted[:5] # NOTE: (year whree most measurements weree takeen, most mausrements taken)

[('Burbank', ([2], 13)),
 ('Los Angeles-North Main Street', ([3, 4], 13)),
 ('Azusa', ([1], 12)),
 ('Pico Rivera #2', ([2], 12)),
 ('Santa Clarita', ([2], 12))]

In [10]:
dates = [i[0]+':'+i[1] for i in metadata['dates']]
codes = [datafetcher.all_codes.loc[code]['value_represented'] for code in metadata['codes']]

ndf = pd.DataFrame(data['Los Angeles-North Main Street'])
ndf.index = codes
ndf.columns = dates
ndf # NOTE: Table for when we have values in chosen station!

,20000528:20000529,20051030:20051031,20100301:20100302,20150413:20150414,20200702:20200703
Carbon monoxide,True,True,True,True,True
Nitrogen dioxide (NO2),True,True,True,True,True
Ozone,True,True,True,True,True
PM2.5 - Local Conditions,True,True,True,True,True
Wind Direction - Resultant,True,True,False,True,True
Wind Speed - Resultant,True,True,False,True,True
Outdoor Temperature,True,True,True,True,True
Relative Humidity,True,True,True,True,True
Solar radiation,True,False,False,True,True
Ultraviolet radiation,False,False,False,True,True


From the results above we will proceed with <b>Los Angeles-North Main Street</b> as our primary location to model. Notice that it has data on all our criteria pollutants and MET variables. We now proceed to find the amount of VOC data we have for these sites with the same sampling as before.

<b>SAVE CODES AND YEARS IN DICT TOO, USE DATES TO SEARCH FOR VOC</b>

In [11]:
# Pick 5 best sites
best_sites = [i[0] for i in yearly_best_sorted[:5]]
best_sites_codes = [datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value=i, nparams={'state':'06', 'county':'037'}) for i in best_sites]
best_sites_dates = [[metadata['dates'][j] for j in i[1][0]] for i in yearly_best_sorted[:5]]
best_sites

['Burbank',
 'Los Angeles-North Main Street',
 'Azusa',
 'Pico Rivera #2',
 'Santa Clarita']

In [12]:
voc_r = datafetcher.find_voc_availability(best_sites, best_sites_codes, best_sites_dates)
with open('voc_data.json', 'w') as f:
    json.dump(voc_r, f)

# with open('voc_data.json', 'r') as f:
#   voc_r = json.load(f)

Finished site 1002, Burbank
Finished site 1103, Los Angeles-North Main Street
Finished site 0002, Azusa
Finished site 1602, Pico Rivera #2
Finished site 6012, Santa Clarita


In [13]:
voc_data = np.array(voc_r['Data'])
voc_df = pd.DataFrame(voc_r['Data'])
voc_df.index = voc_r['Metadata']['codes']
voc_df.head(5)

,Burbank,Los Angeles-North Main Street,Azusa,Pico Rivera #2,Santa Clarita
43000,[False],"[False, True]",[True],[False],[False]
43102,[False],"[False, True]",[True],[False],[False]
43202,[False],"[False, True]",[True],[False],[False]
43203,[False],"[False, True]",[True],[False],[False]
43204,[False],"[False, True]",[True],[False],[False]


In [14]:
voc_site_results = {}
for site in voc_r['Data']:
    arr = np.array(voc_r['Data'][site])
    voc_site_results[site] = arr.sum(axis=0)
voc_site_results

{'Burbank': array([-1]),
 'Los Angeles-North Main Street': array([ 0, 59]),
 'Azusa': array([56]),
 'Pico Rivera #2': array([0]),
 'Santa Clarita': array([0])}

In [15]:
voc_r['Metadata']['dates']

[[['20100301', '20100302']],
 [['20150413', '20150414'], ['20200702', '20200703']],
 [['20051030', '20051031']],
 [['20100301', '20100302']],
 [['20100301', '20100302']]]

### Explore dataset for chosen sight W VOC data

Los Angeles-North Main Street had the most CRITERIA, MET, and VOC data (almost all the PAMS_VOCS are in this data set) <b>for the sampled date in 2020</b>

In [16]:
print(yearly_best_sorted[1])
print(best_sites_codes[1])

('Los Angeles-North Main Street', ([3, 4], 13))
1103


In [ ]:
# df = datafetcher.create_dataset(20200101, 20200102, site='1103', county='037', state='06', processed=True, verbose=False, vocs=True) # NOTE: This crashes because my kernel on my computer can't handle the computation.
# df.shape

/Users/tcintra/Desktop/research/chem150/chem150/venv/lib/python3.7/site-packages/pandas/core/frame.py:9166: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitude_x', 'latitude_x'} in the result is deprecated and will raise a MergeError in a future version.
  joined, frame, how=how, left_index=True, right_index=True


(192, 13)

### Include Emissions projections for chosen sight W/O VOC data

In [70]:
site_code = datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value='Los Angeles-North Main Street', nparams={'state':'06', 'county':'037'})

# Example of Site data using Los Angeles-North Main Street, Los Angeles, California
df = datafetcher.create_dataset(20180101, 20181231, site=site_code, county='037', state='06', processed=True, verbose=False, vocs=False)

/Users/tcintra/Desktop/research/chem150/chem150/venv/lib/python3.7/site-packages/pandas/core/frame.py:9166: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitude_x', 'latitude_x'} in the result is deprecated and will raise a MergeError in a future version.
  joined, frame, how=how, left_index=True, right_index=True


In [71]:
# NOTE: The (lat, lon) for LA North Main St is (34.07, -118.23)
# NOTE: The HEMCO CEDS netcdf files use bounding box every 50 km, so we want the closest box (34.25, -118.25)
print(df.shape)
df

(8760, 13)


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure,latitude,longitude
datetime,,,,,,,,,,,,,
2018-01-01 00:00:00,1.4490,27.2,0.002,61.4,49.0,3.2,51.8,87.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.0,0.0,1009.0,34.06659,-118.22688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.6230,32.4,0.007,NaN,32.0,1.8,55.4,15.0,0.0,0.0,999.0,34.06659,-118.22688
2018-12-31 20:00:00,0.8230,34.0,0.003,NaN,25.0,2.0,52.4,19.0,0.0,0.0,1000.0,34.06659,-118.22688
2018-12-31 21:00:00,0.7425,31.7,0.005,NaN,30.0,2.3,51.7,18.0,0.0,0.0,1001.0,34.06659,-118.22688


In [77]:
nc_links, _ = datafetcher.get_ceds_links(year='2018')
la_lat = 34.25
la_lon = -118.25
ceds_df = DataFetcher.make_ceds_df(la_lat, la_lon, nc_links)
ceds_df

,ALD2_agr,ALD2_ene,ALD2_ind,ALD2_rco,ALD2_shp,ALD2_slv,ALD2_tra,ALD2_wst,ALK4_butanes_agr,ALK4_butanes_ene,...,TOLU_tra,TOLU_wst,XYLE_agr,XYLE_ene,XYLE_ind,XYLE_rco,XYLE_shp,XYLE_slv,XYLE_tra,XYLE_wst
2018-01-01 00:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 01:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 02:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 03:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 04:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 20:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 21:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 22:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13


In [78]:
big_df =df.join(ceds_df, how='inner')
print(big_df.shape)
big_df

(8760, 261)


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,...,TOLU_tra,TOLU_wst,XYLE_agr,XYLE_ene,XYLE_ind,XYLE_rco,XYLE_shp,XYLE_slv,XYLE_tra,XYLE_wst
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,1.4490,27.2,0.002,61.4,49.0,3.2,51.8,87.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.6230,32.4,0.007,NaN,32.0,1.8,55.4,15.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 20:00:00,0.8230,34.0,0.003,NaN,25.0,2.0,52.4,19.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 21:00:00,0.7425,31.7,0.005,NaN,30.0,2.3,51.7,18.0,0.0,0.0,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13


## Missing data

In [67]:
# Check if we are missing data for any measurement
df[df.isna().any(axis=1)]

# TODO: interpolate

,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure,latitude,longitude
datetime,,,,,,,,,,,,,
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.00,0.00,1009.0,34.06659,-118.22688
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.00,0.00,1009.0,34.06659,-118.22688
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.00,0.00,1009.0,34.06659,-118.22688
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.00,0.00,1009.0,34.06659,-118.22688
2018-01-01 05:00:00,1.4360,28.0,0.001,NaN,35.0,4.3,48.7,79.0,0.00,0.00,1009.0,34.06659,-118.22688
2018-01-01 06:00:00,1.3560,27.1,0.001,NaN,38.0,3.8,48.0,78.0,0.00,0.00,1009.0,34.06659,-118.22688
2018-01-01 07:00:00,1.2335,28.7,0.002,NaN,33.0,4.3,49.3,73.0,0.04,0.00,1009.0,34.06659,-118.22688
2018-01-01 08:00:00,1.0725,31.5,0.007,NaN,37.0,3.4,53.8,59.0,0.24,0.01,1010.0,34.06659,-118.22688
2018-01-01 09:00:00,1.0710,33.6,0.015,NaN,47.0,2.0,58.7,49.0,0.46,0.02,1011.0,34.06659,-118.22688


In [79]:
df.to_csv('./data/sample.csv')